In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torchsummary import summary

In [2]:
# Define transformations for training and validation data
train_transforms = transforms.Compose([
    transforms.Resize((480, 270)),
    transforms.ToTensor(),
])

val_transforms = transforms.Compose([
    transforms.Resize((480, 270)),
    transforms.ToTensor(),
])

In [3]:
# Load training and validation data
train_data = datasets.ImageFolder('data/train', transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=96, shuffle=True)

val_data = datasets.ImageFolder('data/test', transform=val_transforms)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=96, shuffle=False)

# Define the model - Load pre-trained VGG16 equivalent to weights=VGG16_Weights.IMAGENET1K_V1
model = models.vgg16(weights='VGG16_Weights.DEFAULT')

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

# Modify the last layer for our task
model.classifier[-1] = nn.Linear(4096, 4)  # 4 outputs - forward, left, right and brake

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Moving the model to Gpu 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# print(model)
summary(model, input_size=(3 ,480, 270))#Show the model(Resnet50) Structure details 

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 480, 270]           1,792
              ReLU-2         [-1, 64, 480, 270]               0
            Conv2d-3         [-1, 64, 480, 270]          36,928
              ReLU-4         [-1, 64, 480, 270]               0
         MaxPool2d-5         [-1, 64, 240, 135]               0
            Conv2d-6        [-1, 128, 240, 135]          73,856
              ReLU-7        [-1, 128, 240, 135]               0
            Conv2d-8        [-1, 128, 240, 135]         147,584
              ReLU-9        [-1, 128, 240, 135]               0
        MaxPool2d-10         [-1, 128, 120, 67]               0
           Conv2d-11         [-1, 256, 120, 67]         295,168
             ReLU-12         [-1, 256, 120, 67]               0
           Conv2d-13         [-1, 256, 120, 67]         590,080
             ReLU-14         [-1, 256, 

In [4]:
# # Move model to GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# Training loop
epochs = 25  # You can adjust this
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = (correct / total) * 100

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = (correct / total) * 100

    print(f"Epoch {epoch + 1}/{epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")



Epoch 1/25, Train Loss: 1.0581, Train Acc: 54.25%, Val Loss: 0.8386, Val Acc: 66.99%
Epoch 2/25, Train Loss: 0.8232, Train Acc: 67.39%, Val Loss: 0.7242, Val Acc: 74.49%
Epoch 3/25, Train Loss: 0.7604, Train Acc: 70.62%, Val Loss: 0.7038, Val Acc: 73.13%
Epoch 4/25, Train Loss: 0.7291, Train Acc: 72.10%, Val Loss: 0.6494, Val Acc: 76.17%
Epoch 5/25, Train Loss: 0.6921, Train Acc: 72.77%, Val Loss: 0.6274, Val Acc: 76.81%
Epoch 6/25, Train Loss: 0.6910, Train Acc: 73.21%, Val Loss: 0.6130, Val Acc: 76.93%
Epoch 7/25, Train Loss: 0.6697, Train Acc: 73.73%, Val Loss: 0.5859, Val Acc: 78.76%
Epoch 8/25, Train Loss: 0.6773, Train Acc: 74.53%, Val Loss: 0.5721, Val Acc: 79.40%
Epoch 9/25, Train Loss: 0.6409, Train Acc: 73.93%, Val Loss: 0.5617, Val Acc: 78.80%
Epoch 10/25, Train Loss: 0.6435, Train Acc: 73.77%, Val Loss: 0.5477, Val Acc: 79.76%
Epoch 11/25, Train Loss: 0.6440, Train Acc: 74.93%, Val Loss: 0.5435, Val Acc: 79.04%
Epoch 12/25, Train Loss: 0.6158, Train Acc: 75.73%, Val Loss: 0

In [5]:
 # Save the model
torch.save(model.state_dict(), f'models/gta_vgg16_lr{learning_rate}_{epochs}_epochs.pth')